In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
import scipy
from typing import List, Any, Dict, Union, Tuple
import transforms3d as t3d
from scipy import signal
import math
from helpers import vectorize_to_np, rpy_to_pose_mat_np, visualize_3d, parse_record,visualize_1d

In [ ]:
# filter accel
def filter_accel(accel: np.ndarray, band: Tuple[float, float] = (0.005, 0.9999)):
    res = np.copy(accel)
    b, a = signal.butter(7, band, 'bandpass')
    res[:, 0] = signal.filtfilt(b, a, accel[:, 0])
    res[:, 1] = signal.filtfilt(b, a, accel[:, 1])
    res[:, 2] = signal.filtfilt(b, a, accel[:, 2])
    return res


def get_gravity_projection(rpy, local_gravity: np.ndarray = np.array([0, 0, -9.8]), thresh: int = 50):
    _pose_mat = t3d.euler.euler2mat(rpy[:thresh, 0].mean(), rpy[:thresh, 1].mean(), rpy[:thresh, 2].mean())
    g = _pose_mat @ local_gravity  # gravity projected to imu coordinate
    return g


def get_accel_offset(accel: np.ndarray,
                     g: np.ndarray,
                     thresh: int = 100) -> np.ndarray:
    reading = np.mean(accel[:thresh, :], axis=0)
    offset = reading - g
    return offset

## Zero-velocity update policy
- filter of vel
- thresh hold

In [ ]:
def zero_vel_determination(
    vel: np.ndarray,
    gyro: np.ndarray,
    accel: np.ndarray,
    thresh: Tuple[float] = (0.1,0.1,5,0.1)
) -> bool:
    if vel.shape[0] > 0 and gyro.shape[0] > 0 and accel.shape[0] > 0:
        vel_mean = np.sqrt(np.sum(np.mean(vel, axis=0)**2))
        gyro_mean = np.sqrt(np.sum(np.mean(gyro, axis=0)**2))
        gyro_std = np.mean(np.std(gyro, axis=0))
        accel_mean = np.sqrt(np.sum(np.mean(accel, axis=0)**2))
        accel_std = np.mean(np.std(accel, axis=0))
        # print(vel_mean, gyro_mean, gyro_std, accel_mean, accel_std)
        if gyro_mean < thresh[0] and gyro_std < thresh[1] and accel_mean < thresh[2] and accel_std < thresh[3]:
            return True
        else:
            return False
    else:
        return False

In [ ]:
record_name = '/Users/liyutong/projectExchange/imu-python-tools/2021-11-12-19:11:39_cu.usbserial-112420_IMU_record.json'
accel_raw, rpy, gyro, pose_mat, timestamp = parse_record(record_name)

# npzfile = np.load('imu_7cdfa1b9a340-2.npz')
# accel_raw = np.squeeze(np.stack([npzfile['accel_x'], npzfile['accel_y'], npzfile['accel_z']], axis=1)) * 9.8
# gyro = np.squeeze(np.stack([npzfile['gyro_x'], npzfile['gyro_y'], npzfile['gyro_z']], axis=1))
# rpy = np.squeeze(np.stack([npzfile['roll'], npzfile['pitch'], npzfile['yaw']], axis=1)) * np.pi / 180
# mag = np.squeeze(np.stack([npzfile['mag_x'], npzfile['mag_y'], npzfile['mag_z']], axis=1))
# timestamp = npzfile['timestamp']
# pose_mat = rpy_to_pose_mat_np(rpy)


visualize_3d(accel_raw,timestamp, 'ACCE_RAW')
thresh = 0
accel_raw = accel_raw[thresh:]
gyro = gyro[thresh:]
rpy = rpy[thresh:]
pose_mat = pose_mat[thresh:]
timestamp = timestamp[thresh:]
zero_vel = np.zeros_like(timestamp, dtype=np.int64)

In [ ]:
GRAVITY_SHANGHAI = np.array([0,0,-9.7946])
bias = np.array([-1.75098646,  0.10479917,  0.03107138]) # Previous iteration
accel_raw -= bias

accel = np.copy(accel_raw)
# Project gravity to local coordinate, then substract accel initial readings (mesured g) with projected gravity
# assumed_gain = np.array([1,1,1])
g_projection = get_gravity_projection(rpy, GRAVITY_SHANGHAI, 50)
print(f"g_projection={g_projection}")
accel_bias = get_accel_offset(accel, g_projection, 50)
print(f"accel_bias={accel_bias}")
# accel -= accel_bias

# Sustract gravity
gravity = np.empty_like(accel)
for i in range(len(timestamp)):
    gravity[i] = pose_mat[i] @ GRAVITY_SHANGHAI
accel -= gravity

# filter accel
accel = filter_accel(accel, (0.005,0.999))
print(f"accel.mean={np.mean(accel[:thresh,:],axis=0)}")

for i in range(len(timestamp)):
    accel[i] = np.linalg.inv(pose_mat[i]) @ accel[i]

In [ ]:
# calc velocity, with zero velocity update policy
window_sz = 3
vel = np.zeros_like(accel)
for i in range(len(timestamp) - 1):
    if zero_vel_determination(vel[i-window_sz:i,:], gyro[i-window_sz:i,:], accel[i-window_sz:i,:]):
        vel[i + 1] = 0
        zero_vel[i] = 1
    else:
        vel[i + 1] = vel[i] + 0.5 * (accel[i + 1] + accel[i]) * (timestamp[i + 1] - timestamp[i])

# calc displacement
# Mid-value integration
pos = np.zeros_like(accel)
for i in range(len(timestamp) - 1):
    pos[i + 1] = pos[i] + 0.5 * (vel[i + 1] + vel[i]) * (timestamp[i + 1] - timestamp[i])


col = np.ones_like(pos)
col[:, 0] = np.linspace(1, 0, col.shape[0])

print('Calc Finished')

In [ ]:
visualize_3d(pos,timestamp, 'POS')
visualize_3d(accel,timestamp, 'ACCEL')
visualize_3d(gyro,timestamp, 'GYRO')
visualize_3d(rpy,timestamp, 'RPY')
visualize_3d(vel,timestamp, 'VEL')
visualize_3d(gravity,timestamp, 'GRAVITY')
visualize_1d(zero_vel, timestamp, 'ZV')


HTN
FastForward
LGP

In [ ]:
visualize_1d(zero_vel, timestamp, 'ZV')
cali_points = []
for idx, status in enumerate(zero_vel):
    if status > 0:
        cali_points.append({"idx": idx, "mes": accel_raw[idx],"rpy": rpy[idx], "g": gravity[idx]})
        

## Error model

$mes = gain.real + bias + noise$

$noise \sim N(0,\mu)$

$\overline{real} = gain^{-1}.(\overline{mes} - bias)$

In [ ]:

mes = []
real = []
for point in cali_points:
    real_gravity = gravity[point['idx']]
    mes.append(point['mes'])
    real.append(real_gravity)
mes = np.vstack(mes)
real = np.vstack(real)
# Plan1 mes = real + bias + noise

bias = mes.mean(axis=0) - real.mean(axis=0)


In [ ]:
accel[:,0].sum()
accel[:,1].sum()
accel[:,2].sum()

In [ ]:
mes.mean(axis=0)

In [ ]:
real.mean(axis=0)

In [ ]:
bias